# HBP Morphology Viewer web interface

In [1]:
# The autoreload extension is meant for developers.
# It ensures that libraries are automatically reloaded when they are modified.
%load_ext autoreload

The web interface uses a python script <a href="movi_interface.py">movi_interface.py</a> that deals with the communication with the HBP Morphology Viewer. It defines a class 'MoviInterface' which takes two important arguments:
- the name of the variable that it is locally assigned to (used in callbacks)
- the url of the HBP Morphology Viewer,
  - stable version: https://neuroinformatics.nl/HBP/morphology-viewer,
  - dev version: https://neuroinformatics.nl/HBP/morphology-viewer-dev

In [2]:
import movi_interface as movi
import ipywidgets as widgets
%autoreload 2
# moviInterface will open the HBP Morphology Viewer in a new tab
moviInterface = movi.MoviInterface('moviInterface','https://neuroinformatics.nl/HBP/morphology-viewer-dev')
#moviInterface = movi.MoviInterface('moviInterface','http://localhost/HBP/morphology-viewer-dev')

### Example 1: Display a neuron in the HBP Morphology Viewer
Make sure that the morphology file of the `neuronToDisplay`-variable exists on your system, you can download it <a href="../samples/TCneuron_EP36-S1_9-9-17.DAT">here</a>. After running the following cell, you should see the neuron displayed in the Morphology Viewer tab, and get a notification when it is ready in this tab.

In [ ]:
neuronToDisplay = '../samples/TCneuron_EP36-S1_9-9-17.DAT'

with open(neuronToDisplay,'rb') as fp:
  moviCommand = {
    "method":"MoVi.import",
    "params": {
      "name": neuronToDisplay,
      "contents": fp.read()
    }
  }
moviInterface.send(moviCommand);

### Example 2: Convert a neuron from SWC to Neurolucida XML
Make sure that the morphology file of the `neuronToConvert`-variable exists on your system, you can download it <a href="../samples/c10861.CNG.swc">here</a>. After running the following cell, you should see the neuron displayed in the Morphology Viewer tab, get a notification in this tab when it is ready, and find the resulting XML file in your temporary files folder.

**Method 1: Using moviInterface.awaitResponse**

This method sends a command to the Morphology Viewer, then blocks the Python execution flow until a response is received. This method is recommended when the result is needed in the next cell. The downside of it is that notebook execution is stalled if the response fails to arrive. 

In [ ]:
import tempfile,os.path

neuronToConvert = '../samples/c10861.CNG.swc'

def saveConvertedNeuron(response):
  if 'error' in response:
    print('Error in saveConvertedNeuron: {}'.format(response['error']))
    return
  result = response['result']
  convertedNeuronFile = os.path.join(tempfile.gettempdir(),os.path.basename(result['name']))
  isText = type(result['contents']) == str
  with open(convertedNeuronFile,'wt' if isText else 'wb') as fp:
    fp.write(result['contents'])  
  print('Converted neuron file saved as "{}"'.format(convertedNeuronFile))
  return convertedNeuronFile

with open(neuronToConvert,'rb') as fp:
  moviCommand = {
    "method":"MoVi.convert",
    "params": {
      "name": neuronToConvert,
      "contents": fp.read(),
      "toMime": "model/mbf.xml+gzip",
      "doRender": True
    }
  }
  print('Method 1 (wait for response)')
  response = moviInterface.awaitResponse(moviCommand)
  stdout = widgets.Output(layout={'border': '10px solid #FA0'})
  with stdout:
    convertedNeuronFile = saveConvertedNeuron(response)
  display(stdout)

**Method 2: Using moviInterface.send with a callback**

The positive side of this method is that it does not block the Python execution flow. But as a consequence, the result will only be ready *after* all pending Python commands are executed. That means that you cannot use the result of the callback in any of the next cells. 

The callback should be the name of a function in the current scope, and this function should take the rpc-response as a single argument. Also note the use of an 'output widget' in the code below. This ensures that any output generated asynchronously will appear below the cell.

In [ ]:
print('Method 2 (asynchronous callback)')

# To make the task more challenging, the conversion is now 
# inverted, from gzipped XML to gzipped SWC
with open(convertedNeuronFile,'rb') as fp:
  moviCommand = {
    "method":"MoVi.convert",
    "params": {
      "name": convertedNeuronFile,
      "contents": fp.read(),
      "toMime": "model/swc+gzip",
      "doRender": True
    }
  }

# For this method, the use of an output widget is essential.
# Without it, output and errors of the 'saveConvertedNeuron' 
# callback will be invisible.
stdout = widgets.Output(layout={'border': '10px solid #FA0'})

def save(response):
  # Callback function with a single parameter: the query response
  with stdout:
    saveConvertedNeuron(response)

moviInterface.send(moviCommand,'save');
#rpcId = moviInterface.send(moviCommand)
#print(rpcId)
#responseFuture = moviInterface.fetchResponse(rpcId)
#print(responseFuture)
with stdout:
  print('Awaiting response...')
display(stdout)